# Notebook for Creating the Submission File

In [1]:
import pandas as pd
import numpy as np

def analyze_dataframe(df):
    null_entries = {}
    inf_entries = {}

    for col in df.columns:
        null_count = df[col].isnull().sum()
        if null_count > 0:
            null_entries[col] = null_count

        inf_count = ((df[col] == np.inf) | (df[col] == -np.inf)).sum()
        if inf_count > 0:
            inf_entries[col] = inf_count

    # Find the most frequent columns for null and inf values
    most_frequent_null_column = max(null_entries, key=null_entries.get) if null_entries else None
    most_frequent_inf_column = max(inf_entries, key=inf_entries.get) if inf_entries else None

    print(100*'-')
    print("Shape:", df.shape)
    print("Null entries:", null_entries)
    print("Infinite entries:", inf_entries)
    print("Most frequent column with null entries:", most_frequent_null_column)
    print("Most frequent column with infinite entries:", most_frequent_inf_column)
    print(100*'-')

In [2]:
def add_neighboring_wells(df):
  pad_child_count = dict()
  pad_ids = df["pad_id"].values
  for id in pad_ids:
      if id in pad_child_count:
        pad_child_count[id] += 1
      else:
        pad_child_count[id] = 1

  df["num_neighboring_wells"] = df["pad_id"].map(pad_child_count)
  return df

def euclid_surface_bh_dist(df):
    df['surface_bottom_dist'] = ((df['surface_x'] - df['bh_x'])**2 + (df['surface_y'] - df['bh_y'])**2)**0.5
    return df

def euclid_toe_dist(df):
    df['toe_dist'] = ((df['horizontal_midpoint_x'] - df['horizontal_toe_x'])**2 + (df['horizontal_midpoint_y'] - df['horizontal_toe_y'])**2)**0.5
    return df

def surface_bottom_angle(df):
   df['surface_bottom_angle'] = np.arctan2(df['surface_y'] - df['bh_y'], df['surface_x'] - df['bh_x'])
   return df

def toe_angle(df):
   df['toe_angle'] = np.arctan2(df['horizontal_midpoint_y'] - df['horizontal_toe_y'], df['horizontal_midpoint_x'] - df['horizontal_toe_x'])
   return df

In [3]:
df = pd.read_csv("data_wrangling/datasets/final_unstandardized.csv")

analyze_dataframe(df)

----------------------------------------------------------------------------------------------------
Shape: (17868, 100)
Null entries: {}
Infinite entries: {}
Most frequent column with null entries: None
Most frequent column with infinite entries: None
----------------------------------------------------------------------------------------------------


In [4]:
# Drop the columns that are very empty

columns_to_drop = ['Unnamed: 0']

df = df.drop(columns=columns_to_drop)

analyze_dataframe(df)

----------------------------------------------------------------------------------------------------
Shape: (17868, 99)
Null entries: {}
Infinite entries: {}
Most frequent column with null entries: None
Most frequent column with infinite entries: None
----------------------------------------------------------------------------------------------------


In [5]:
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X = df.drop('OilPeakRate', axis=1)
y = df['OilPeakRate']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(max_depth=40, min_samples_leaf=2, min_samples_split=2, n_estimators=400)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print('RMSE', math.sqrt(mean_squared_error(y_test, y_pred)))

RMSE 99.27393905619753


In [7]:
print(type(y_pred))
print(y_pred)
      

<class 'numpy.ndarray'>
[249.69735515 271.07075024 205.30827688 ... 285.68425941 367.21581171
 174.99605768]


# Train on all

In [8]:
rf.fit(X, y)

RandomForestRegressor(max_depth=40, min_samples_leaf=2, n_estimators=400)

# Prepare Submission DF

In [11]:
scoring_df = pd.read_csv("data_wrangling/datasets/scoring.csv")

analyze_dataframe(scoring_df)

----------------------------------------------------------------------------------------------------
Shape: (400, 30)
Null entries: {}
Infinite entries: {}
Most frequent column with null entries: None
Most frequent column with infinite entries: None
----------------------------------------------------------------------------------------------------


In [12]:
# Drop the columns that are very empty

columns_to_drop = ['Unnamed: 0', 'ffs_frac_type', 'average_stage_length',
       'average_proppant_per_stage', 'average_frac_fluid_per_stage',
       'frac_seasoning']

scoring_df = scoring_df.drop(columns=columns_to_drop)

analyze_dataframe(scoring_df)

----------------------------------------------------------------------------------------------------
Shape: (400, 24)
Null entries: {}
Infinite entries: {}
Most frequent column with null entries: None
Most frequent column with infinite entries: None
----------------------------------------------------------------------------------------------------


In [13]:
# Engineer new features based on developed pipeline

add_neighboring_wells(scoring_df)
euclid_surface_bh_dist(scoring_df)
euclid_toe_dist(scoring_df)
surface_bottom_angle(scoring_df)
toe_angle(scoring_df)

analyze_dataframe(scoring_df)

----------------------------------------------------------------------------------------------------
Shape: (400, 29)
Null entries: {}
Infinite entries: {}
Most frequent column with null entries: None
Most frequent column with infinite entries: None
----------------------------------------------------------------------------------------------------


In [14]:
# Drop features that are not necessary: standardized_operator_name, pad_id

uninterpretable_cols = ['standardized_operator_name', 'pad_id']
scoring_df = scoring_df.drop(columns=uninterpretable_cols)

analyze_dataframe(scoring_df)

----------------------------------------------------------------------------------------------------
Shape: (400, 27)
Null entries: {}
Infinite entries: {}
Most frequent column with null entries: None
Most frequent column with infinite entries: None
----------------------------------------------------------------------------------------------------


In [15]:
# One hot encode and save

categorical_cols = ['relative_well_position', 'number_of_stages', 'batch_frac_classification',
                    'well_family_relationship', 'frac_type']

scoring_df = pd.get_dummies(scoring_df, columns = categorical_cols, dtype=int)

analyze_dataframe(scoring_df)

----------------------------------------------------------------------------------------------------
Shape: (400, 64)
Null entries: {}
Infinite entries: {}
Most frequent column with null entries: None
Most frequent column with infinite entries: None
----------------------------------------------------------------------------------------------------


In [16]:
train_cols = set(X.columns)
new_cols = set(scoring_df.columns)

missing_cols = train_cols - new_cols

for col in missing_cols:
    scoring_df[col] = 0

extra_cols = new_cols - train_cols

scoring_df = scoring_df.drop(columns=extra_cols)

analyze_dataframe(scoring_df)

scoring_df = scoring_df[X.columns]

----------------------------------------------------------------------------------------------------
Shape: (400, 98)
Null entries: {}
Infinite entries: {}
Most frequent column with null entries: None
Most frequent column with infinite entries: None
----------------------------------------------------------------------------------------------------


In [20]:
print(scoring_df.dtypes)

surface_x                                     float64
surface_y                                     float64
bh_x                                          float64
bh_y                                          float64
gross_perforated_length                       float64
                                               ...   
well_family_relationship_Infill Child Well      int64
well_family_relationship_Sibling Well           int64
well_family_relationship_Standalone Well        int64
well_family_relationship_Unknown                int64
frac_type_Primary Frac                          int64
Length: 98, dtype: object


# Generate Submission File

In [25]:
!pip install openpyxl

y_pred = rf.predict(scoring_df)
print(y_pred)
submission = pd.DataFrame(y_pred, columns=['OilPeakRate'])

submission.to_excel("submission_file.xlsx", index=False)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 8.7 MB/s eta 0:00:00
[116.89988932 141.48870475 148.44657702 121.48937978  87.65856388
 171.79196449  92.41328817 113.92552317 132.30608945 152.19997399
 148.36862909 142.5330542   94.28805194 163.68936459 229.26640071
 154.34641654 100.9432596   77.4230554  116.68951343 176.17394667
  79.67459283 177.89261613 148.86596736 259.52864859 258.71652996
 102.41716876 150.6902819  218.01086955 203.5378603  172.44673691
 202.57491461 147.08838432 135.08117402 141.3532594   70.00314372
 219.93508434  84.48118586 139.05408046 248.22828516 204.57779854
 218.01208454 307.98636733 223.5707961  252.64459676 213.86905041
 267.36306777 406.48360515 221.59724237 340.65542223 360.85642032
 277.64155241 301.69524877 407.06143278 155.49259431 213.88970655
 363.72487755 132.80827821 225.95531747 390.13665537 128.85419242
 249.97533608 118.40283306 332.67468116 222.37422181 219.9257497
 278.72955145 263.48600544 275.56648804 126.30400738 131.61810